# AI-driven Interactive Metadata 

*Amanda Birmingham, Dept. of Pediatrics, UC San Diego*

A natural-language approach to metadata investigation and cleaning using the `ChatGPT 4` LLM, the `Langchain` AI framework, AI-based speech recognition, and interactive `itables` visualization.

## Initial set-up

To be performed outside notebook:

## Adjustable LLM settings

In [1]:
g_GEMINI = "GOOGLE_API_KEY"
g_CHATGPT = "OPENAI_API_KEY"

g_chosen_llm = g_CHATGPT

In [2]:
g_use_speech = True

In [3]:
# WARNING: Increasing this number will increase the amount of information included in each 
# LLM query and thus increase the cost of the queries!
# Decreasing this number will make the LLM forget past exchanges more quickly
g_num_msgs_in_history = 4

In [4]:
g_base_prompt = f"using pandas 3 and python 3.10+ to clean data in jupyter lab."
g_ds_prompt = f"You are a data scientist {g_base_prompt}"
g_pf_prompt = f"You are a professor of data science teaching a class on {g_base_prompt}." 

## Imports

In [5]:
import os
import pandas as pd
import re

In [6]:
import ipywidgets as widgets
from IPython import get_ipython
from IPython.display import display
from traitlets import observe, link, Unicode, Bool, Any
from itables import init_notebook_mode, show
from ipylab import JupyterFrontEnd
import time

In [7]:
import speech_recognition as speech_recog

In [8]:
from langchain_core.messages import trim_messages
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

In [9]:
import warnings
# warnings.filterwarnings('ignore')

In [10]:
import logging
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(level=logging.WARNING, format='%(levelname)s: %(message)s')

In [11]:
import qiimp 

## Model creation

In [12]:
assert g_chosen_llm in os.environ, f"Please set the {g_chosen_llm} environment variable."

In [13]:
if g_chosen_llm == g_CHATGPT:
    from langchain_openai import ChatOpenAI
    _g_chat_model = ChatOpenAI(model="gpt-4o-mini")
elif g_chosen_llm == g_GEMINI:
    from langchain_google_genai import ChatGoogleGenerativeAI
    _g_chat_model = ChatGoogleGenerativeAI(
        model="gemini-1.5-pro",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
    )
else:
    raise ValueError(f"Unrecognized llm model '{g_chosen_llm}'")

## Chat creation

In [14]:
def _start_chat(model, custom_prompt):    
    # Define trimmer
    # count each message as 1 "token" (token_counter=len) and 
    #keep only the last x messages
    trimmer = trim_messages(strategy="last", max_tokens=g_num_msgs_in_history, 
                            token_counter=len)
    
    workflow = StateGraph(state_schema=MessagesState)
    
    # Define the function that calls the model
    def call_model(state: MessagesState):
        trimmed_messages = trimmer.invoke(state["messages"])
        system_prompt = custom_prompt
        messages = [SystemMessage(content=system_prompt)] + trimmed_messages
        response = model.invoke(messages)
        return {"messages": response}
    
    
    # Define the node and edge
    workflow.add_node("model", call_model)
    workflow.add_edge(START, "model")
    
    # Add simple in-memory checkpointer
    memory = MemorySaver()
    app = workflow.compile(checkpointer=memory)
    return app

In [15]:
_g_chat = _start_chat(_g_chat_model, g_ds_prompt)

## Front-end helpers

In [16]:
_g_front_end = JupyterFrontEnd()

In [17]:
def _insert_and_populate(statement=None, move_up=True):
    _g_front_end.commands.execute('notebook:insert-cell-below')
    if statement is not None:
        time.sleep(0.2)
        _g_front_end.commands.execute('notebook:replace-selection', { 'text': statement})
        _g_front_end.commands.execute('notebook:enter-edit-mode')

    if move_up:
        _g_front_end.commands.execute('notebook:move-cell-up') 
        

def _insert_and_run(statement=None, move_up=True):
    _insert_and_populate(statement, move_up)
    
    if statement is not None:
        _g_front_end.commands.execute('notebook:run-cell-and-select-next')  
        _g_front_end.commands.execute('notebook:enter-edit-mode')

## Prompt engineering

In [18]:
# ai helper prompts
g_unique_set_prefix = "for column named"
g_col_check_prefix = "check column named"
g_code_prefix = "write code to"

# non-ai prompts
g_summarize_statement = "summarize table"

# button prompts
g_add_cell_statement = "add cell"
g_copy_last_statement = "copy it"
g_run_last_statement = "now run it"
g_revert_df_statement = "revert dataframe"

In [19]:
def _change_first_char(a_str, upper=True):
    if upper:
        a_lambda = lambda x: x.groups()[0].upper()
    else:
        a_lambda = lambda x: x.groups()[0].lower()
        
    # affect ONLY first letter, leave all the rest alone
    # (so different that a_str.capitialize() or a_str.title())
    return re.sub('([a-zA-Z])', a_lambda, a_str, 1)   


def _expand_prompt(user_prompt, df_name, a_df):
    df_prompt = f"You are given the '{df_name}' dataframe with columns {list(a_df.columns)}. "
    
    if user_prompt.startswith(g_col_check_prefix):
        col_name = user_prompt.replace(g_col_check_prefix, "").strip().replace(" ", "_")
        user_prompt = _get_explore_col_prompt(col_name)     
    # end if starts with check prefix--which itself adds a unique set prefix
    
    if user_prompt.startswith(g_unique_set_prefix):
        new_prompt = _get_unique_set_prompt(user_prompt, a_df)
    elif user_prompt.startswith(g_code_prefix):
        new_prompt = _get_code_prompt(user_prompt)        
    else:
        new_prompt = user_prompt

    new_prompt = df_prompt + new_prompt

    return new_prompt


def _get_explore_col_prompt(col_name):
    explore_prompt = "suggest the appropriate Pandas data type for the values in this column, very briefly hypothesize about what they represent, and indicate if any look invalid or unexpected." # Please keep responses short and to the point."
    col_prompt = f"{g_unique_set_prefix} {col_name}, {explore_prompt}"
    return col_prompt


def _get_unique_set_prompt(user_prompt, a_df):
    err_msg = f"Please phrase your question as {g_unique_set_prefix} <col_name>, <question about unique values of column>"
    
    if not user_prompt.startswith(g_unique_set_prefix):
        return err_msg

    prompt_split = user_prompt.split(",")
    if len(prompt_split) < 2:
        return err_msg

    col_name = prompt_split[0].replace(g_unique_set_prefix, "").strip()
    new_prompt = f"For the column named '{col_name}' containing the set of values  {set(a_df[col_name])}, {','.join(prompt_split[1:])}"
    return new_prompt


def _get_code_prompt(user_prompt):
    err_msg = f"Please phrase your question as {g_code_prefix} <perform some operation>"
    
    if not user_prompt.startswith(g_code_prefix):
        return err_msg

    new_prompt = f"{user_prompt} Do not include any non-comment explanations, import statements, or the instantiation of the dataframe. Do not include markdown formatting in your output. Provide runnable code as output."
    return new_prompt

## State management

In [20]:
_g_last_code_out = {}
_g_last_working_df = {}

In [21]:
_g_LAST_CODE_NAME = "AI-generated code"
_g_LAST_DF_NAME = "g_working_df"   


def _save_state(state_dict, obj_to_save, state_name, use_last_execution_num=False):
    # this does NOT copy the input obj_to_save before saving it--that should happen outside this call, if needed
    execution_num = get_ipython().execution_count
    if use_last_execution_num:
        execution_num = execution_num - 1
    
    if execution_num in state_dict:
        warnings.warn(f"{state_name} already contains state for a cell with execution number {execution_num}, which will be overwritten.")
    state_dict[execution_num] = obj_to_save
    return state_dict


def _get_last_state(state_dict):
    last_value = None
    if state_dict is not None and len(state_dict)>0:
        last_key, last_value = next(reversed(state_dict.items()))
    return last_value


def _df_changed():
    #print("in _df_changed")
    last_working_df = _get_last_state(_g_last_working_df)

    if last_working_df is not None:
        #print(f"g_working_df cols: {g_working_df.columns}")
        #print(f"last_working_df cols: {last_working_df.columns}")
        if not last_working_df.equals(g_working_df):
            #print("are different")
            return True
    return False


def _revert_df():
    global g_working_df
    msg = f"There is no {_g_LAST_DF_NAME} state stored." 

    last_working_df = _get_last_state(_g_last_working_df)
    if last_working_df is not None:
        if _df_changed():
            store_working_df()
            g_working_df = last_working_df
            msg = f"{_g_LAST_DF_NAME} reverted to last saved state."
        else:
            msg = f"{_g_LAST_DF_NAME} has not changed since last saved state."
    return msg


def store_working_df(a_df=None, use_last_execution_num=False):
    if a_df is None:
        a_df = g_working_df.copy()
    _save_state(_g_last_working_df, a_df, _g_LAST_DF_NAME, use_last_execution_num=use_last_execution_num)


def revert_df():
    statement = _revert_df()
    return statement


# decorator
def stateful(func):
    def wrapper(*args, **kwargs):
        if _df_changed():
            store_working_df(use_last_execution_num=True)
        func(*args, **kwargs)
    return wrapper

## Chat helper function creation

In [22]:
# Instantiating an empty notebook allows prompt methods to bind to the 
# variable so the can use the real contents later without needing to be 
# passed an argument
g_working_df = pd.DataFrame()

In [23]:
def _summarize_col(col_name, a_df=None, max_items_shown=10):
    a_df = a_df if a_df is not None else g_working_df
    
    lines = []
    lines.append(f"{col_name}")
    lines.append("================")
    a_col = a_df[col_name]
    
    summary = []
    col_uniques = a_col.unique()
    count_uniques = len(col_uniques)
    if a_col.is_unique:
        summary.append(f"All {count_uniques} values are unique.")
    else: 
        summary.append(f"There are {count_uniques} unique value(s) in {len(a_col)} total values.")
    # end if all are unique
        
    caveat = f"first {max_items_shown} " if count_uniques > max_items_shown else ""
    summary.append(f"The {caveat}unique value(s):{col_uniques[:max_items_shown]}.")
    lines.append(" ".join(summary))

    lines.append(f"The current datatype is {a_col.dtype}.")
    lines.append(" ")
    return lines


@stateful
def summarize_col(col_name, a_df=None):
    result = _summarize_col(col_name, a_df)
    result_str = "\n".join(result)
    print(result_str)    


@stateful
def summarize(a_df=None, max_items_shown=10):
    a_df = a_df if a_df is not None else g_working_df
    
    result = [f"The dataframe has {len(a_df)} rows and {len(a_df.columns)} columns.", " "]
    for curr_col_name in a_df.columns:
        curr_result = _summarize_col(curr_col_name, a_df)
        result.extend(curr_result)
    # next column
    result_str = "\n".join(result)
    print(result_str)


@stateful
def find_problem_headers(a_df=None):
    a_df = a_df if a_df is not None else g_working_df

    invalid_cols = a_df.columns[a_df.columns.str.contains(r'[^a-zA-Z0-9._ ]', regex=True)]
    if len(invalid_cols) == 0:
        out = "No invalid column headers found."
    else:
        out = invalid_cols
    print(out)


@stateful
def scrub_headers(a_df=None, lcase_headers=True):
    a_df = a_df if a_df is not None else g_working_df

    a_df.columns = a_df.columns.str.replace(r'[^a-zA-Z0-9]', '_', regex=True)  
    a_df.columns = a_df.columns.str.replace(r'__+', '_', regex=True)  
    a_df.columns = a_df.columns.str.strip('_')
    if lcase_headers:
        a_df.columns = a_df.columns.str.lower()
    print(a_df.columns)


@stateful
def find_problem_records(a_df=None):
    a_df = a_df if a_df is not None else g_working_df

    # get records with leading or trailing spaces in any field
    problem_records = a_df[a_df.apply(lambda x: x.str.contains(r'^\s|\s$', na=False).any(), axis=1)]
    if len(problem_records) == 0:
        print("No problem records found.")
    else:
        display(problem_records)
        return problem_records


@stateful
def scrub_problem_records():
    global g_working_df
    
    # Remove leading or trailing spaces from any field in the dataframe
    g_working_df = g_working_df.map(lambda x: x.strip() if isinstance(x, str) else x)
    print("Problem records scrubbed.")

In [24]:
def _ask(user_prompt, df_name=None, a_df=None):
    df_name = df_name if df_name is not None else "g_working_df"
    a_df = a_df if a_df is not None else g_working_df
    
    new_prompt = _expand_prompt(user_prompt, df_name, a_df)    
    result = _g_chat.invoke(
        {"messages": [HumanMessage(content=new_prompt)]},
        config={"configurable": {"thread_id": "1"}},
    )
    return new_prompt, result


def _get_result_text(a_result):
    return a_result.get("messages")[-1].content


def _clean_answer(answer_str):
    answer_str = re.sub("^```python\n", "", answer_str)
    answer_str = re.sub("\n```$", "", answer_str)   
    return answer_str
    

def _explore_col(col_name, df_name=None, a_df=None):
    full_prompt = _get_explore_col_prompt(col_name)
    return _ask(full_prompt, df_name, a_df)


def _run_non_ai_prompts(user_prompt):
    ai_needed = False
    
    # Note, these have to be full prompts, not prompt prefixes
    if user_prompt == g_summarize_statement:
        summarize()
    else:
        ai_needed = True
    return ai_needed
    

@stateful
def explore_col(col_name, df_name=None, a_df=None):
    _, result = _explore_col(col_name, df_name, a_df)
    print(_get_result_text(result))


@stateful
def ask(user_prompt, df_name=None, a_df=None, show_prompt=False):  
    orig_prompt = user_prompt

    try:
        user_prompt = _change_first_char(user_prompt, upper=False)
        ai_needed = _run_non_ai_prompts(user_prompt)
    
        if ai_needed:
            # otherwise, ask AI
            prompt, result = _ask(user_prompt, df_name, a_df)  
            output = _get_result_text(result)
            output = _clean_answer(output)
        
            if show_prompt:
                output = prompt + "\n\n" + output
            _save_state(_g_last_code_out, output, _g_LAST_CODE_NAME)
            print(output)
    #except Exception as ex:
    #    print(f"I don't understand the prompt:\n{orig_prompt}")
    finally:
        pass

## Widget creation

In [25]:
# from https://github.com/jupyter-widgets/ipywidgets/issues/2962#issuecomment-724210454
class ConfirmationButton(widgets.HBox):
    button_style = Any(default_value='')
    description = Unicode()
    disabled = Bool()
    icon = Unicode()
    layout = Any()
    style = Any()
    tooltip = Unicode()
    
    def  __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._button = widgets.Button(**kwargs)
        self._confirm_btn = widgets.Button(description='Confirm', icon='check', 
                                           button_style='success', layout=dict(width='auto'))
        self._cancel_btn = widgets.Button(description='Cancel', icon='times', 
                                          button_style='warning', layout=dict(width='auto'))
        self._button.on_click(self._on_btn_click)
        self._cancel_btn.on_click(self._on_btn_click)
        self._confirm_btn.on_click(self._on_btn_click)
        self.children = [self._button]
        for key in self._button.keys:
            if key[0]!='_':
                link((self._button,key), (self, key))
        
    def on_click(self, *args, **kwargs):
        self._confirm_btn.on_click(*args, **kwargs)
        
    def _on_btn_click(self, b):
        if b==self._button:
            self.children = [self._confirm_btn, self._cancel_btn]
        else:
            self.children = [self._button]

In [26]:
def _add_cell(a_button=None):
    #_g_front_end.commands.execute('notebook:insert-cell-below')
    #_g_front_end.commands.execute('notebook:move-cell-up')  
    _insert_and_run(None, move_up=True)
    

def _revert_df_from_button(a_button=None, move_up=True):
    statement = revert_df()
    _insert_and_run(statement, move_up=move_up) 


def _copy_or_run_suggestion(a_button, move_up, copy_only=True):
    statement = _get_last_state(_g_last_code_out)
    statement = statement if statement else f'There is no {_g_LAST_CODE_NAME} stored.' 
    if copy_only:
        _insert_and_populate(statement, move_up=move_up)
    else:
        _insert_and_run(statement, move_up=move_up)    


@stateful
def copy_suggestion(a_button=None, move_up=True):
    _copy_or_run_suggestion(a_button, move_up, copy_only=True)


@stateful
def run_suggestion(a_button=None, move_up=True):
    _copy_or_run_suggestion(a_button, move_up, copy_only=False)

In [27]:
def _add_first_button(a_widget, a_button):
    curr_buttons = list(a_widget.children)
    curr_buttons.insert(0, a_button)
    return tuple(curr_buttons)  

In [28]:
_g_add_cell_button = widgets.Button(
    description="Add Cell",
    button_style="primary",  # full blue
    tooltip="Add an empty code cell",
    icon="plus"
)
_g_add_cell_button.on_click(_add_cell)


_g_copy_suggestion_button = widgets.Button(
    description="Copy Suggestion",
    tooltip="Copy AI-generated code to a new cell",
    icon="copy"
)
_g_copy_suggestion_button.style.button_color = 'lightgreen'
_g_copy_suggestion_button.on_click(copy_suggestion)


_g_run_suggestion_button = widgets.Button(
    description="Run Suggestion",
    button_style="info",  # light blue
    tooltip="Run last AI-generated code in a new cell",
    icon="run"
)
_g_run_suggestion_button.on_click(run_suggestion)

_g_undo_button = ConfirmationButton(
    description='Revert Df', 
    tooltip="Revert dataframe to last stored state",
    button_style="warning"  # red
)
_g_undo_button.on_click(_revert_df_from_button)


g_buttons = widgets.HBox([_g_add_cell_button, _g_copy_suggestion_button, _g_run_suggestion_button, _g_undo_button])

In [29]:
_g_unrecognized_msg = "I didn't catch that."

if g_use_speech:
    _g_speech_recognizer = speech_recog.Recognizer()
    
    # lifted from https://youtu.be/2kSPbH4jWME
    _g_record_button = widgets.Button(
        description="Record",
        disabled=False,
        button_style="success",  # full green
        icon="microphone"
    )
    
    def _record_audio(a_button):
        a_button.description = "Recording"
        with speech_recog.Microphone() as source: 
            # listen for 10 seconds for speech to start, 
            # listen for 10 seconds after speech pauses for it to restart
            audio = _g_speech_recognizer.listen(source, 10, 10)
        
        try:
            txt = _g_speech_recognizer.recognize_google(audio)
        except speech_recog.UnknownValueError:
            txt = f"print('{_g_unrecognized_msg}')"
    
        a_button.description = "Record"

        if txt == g_add_cell_statement:
            _add_cell()
        elif txt == g_run_last_statement:
            run_suggestion()
        elif txt == g_revert_df_statement:
            _revert_df()
        else:
            statement = f"ask('{txt}')"
            _insert_and_run(statement)


    _g_record_button.on_click(_record_audio)
    g_buttons.children = _add_first_button(g_buttons, _g_record_button)
# end if use speech

## Dataframe helpers

In [30]:
g_TAB_SEP = "tab"
g_COMMA_SEP = "comma"

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


def load_df(fp, sep_name=g_TAB_SEP, dtype="str", override=False):
    global g_working_df
    if not override:
        proposed_sep = None
        if fp.endswith(".csv") and sep_name==g_TAB_SEP:
            proposed_sep = g_COMMA_SEP
        elif (fp.endswith(".txt") or fp.endswith(".tsv")) and sep_name==g_COMMA_SEP:
            proposed_sep = g_TAB_SEP
    
        if proposed_sep is not None:
            msg = (f"Are you sure this file shouldn't be loaded with a {proposed_sep}?\n"
                   f"If it should be, rerun `load_df` with the `sep_name` parameter "
                   f"set to {proposed_sep}.\n"
                   f"If not, you can run `load_df` with the `override` parameter "
                   f"set to True.")
            print(msg)
            return

    real_sep = None
    if sep_name == g_TAB_SEP:
        real_sep = "\t"
    elif sep_name == g_COMMA_SEP:
        real_sep = ","
    else:
        msg = (f"'{sep_name}' is an unrecognized separator type.  Please choose one of the "
               f" following recognized separators: {[g_TAB_SEP, g_COMMA_SEP]}.")
        print(msg)
        return 

    loaded_df = pd.read_csv(fp, sep=real_sep, dtype=dtype)

    if not override:
        if len(g_working_df) > 0:
            msg = ("This load will overwrite the current contents of g_working_df.\n"
                   "If you don't want to load these contents, copy g_working_df to another "
                   "dataframe variable before running this.\n"
                   "If you really don't care, rerun `load_df` with the `override` "
                   "parameter set to True.")
            print(msg)
            return
            
    g_working_df = loaded_df
    store_working_df()
    return g_working_df
    #display_df()


# conda install dtale -c conda-forge
#import dtale
#d = dtale.show(g_working_df, host='localhost', hide_drop_rows=True, hide_header_editor=True, allow_cell_edits=False, hide_column_menus=True)
#d

## Interactive investigation

To talk to the AI, either type your request within the function `ask()` or, if voice control is enabled, clicking the record button (which will call the `ask` function with your spoken input).

Special statements:
* `check column named <column name>`: asks the AI to draw summarized conclusions about the column and its contents.
    * Example: `ask('check column named time initiate breast')`
* `write code to <description of action>`: asks the AI to limit its responses to code and comments only.
    * Example: `ask('write code to replace within one hour of birth with less one hour')`
* `now run it`: asks the AI to run the last code it wrote

In [31]:
external_metadata_fp = "/Users/abirmingham/Desktop/trpca/15612_expanded_sample_info_10112024_PRJNA277905.csv"
qiita_metadata_fp = "/Users/abirmingham/Desktop/trpca/15612_20240714-052306.txt"
study_config_fp = '/Users/abirmingham/Desktop/trpca/trpca_study.yml'

#### Examine the external metadata

In [32]:
load_df(external_metadata_fp, sep_name="comma")

,LibraryID,Gender,Year of Birth,Ethnicity,Country of Birth,Weight (kg),Height (cm),Alcohol,Passive Smoking,Presence of pets,FLG Genotype,Blomia tropicalis (dust mite),Dermatophagoides pteronyssinus (dust mite),Elaeis guineensis (oil palm pollen),Curvularia spp. (fungus),Skin Prick Test (≥3+),Asthma Status,AR Status,AD Status,Sample Type,Sampling_Area,Sampling_Method
0,WBE005,M,1986,Chinese,Indonesia,Not collected,Not collected,Non-drinker,No,No,WT,0,0,0,0,No,Control,Control,Control,Control,Antecubital fossa,tape
1,WBE006,M,1986,Chinese,Indonesia,Not collected,Not collected,Non-drinker,No,No,WT,0,0,0,0,No,Control,Control,Control,Control,Antecubital fossa,tape
2,WBE007,F,1987,Chinese,Singapore,47,Not collected,Non-drinker,No,No,WT,0,0,0,0,No,Indeterminate,Indeterminate,Control,Control,Antecubital fossa,tape
3,WBE008,F,1987,Chinese,Singapore,47,Not collected,Non-drinker,No,No,WT,0,0,0,0,No,Indeterminate,Indeterminate,Control,Control,Antecubital fossa,tape
4,WBE017,M,1988,Chinese,China,63,178,Occasionally,Yes,No,S1515X,0,0,0,0,No,Control,Indeterminate,Control,Control,Antecubital fossa,tape
5,WBE018,M,1988,Chinese,China,63,178,Occasionally,Yes,No,S1515X,0,0,0,0,No,Control,Indeterminate,Control,Control,Antecubital fossa,tape
6,WBE019,M,1986,Chinese,Singapore,53,163,Occasionally,No,0,WT,0,0,0,0,No,Control,Indeterminate,Control,Control,Antecubital fossa,tape
7,WBE020,M,1986,Chinese,Singapore,53,163,Occasionally,No,0,WT,0,0,0,0,No,Control,Indeterminate,Control,Control,Antecubital fossa,tape
8,WBS016,F,1993,Chinese,China,47,162,Non-drinker,No,No,WT,0,0,0,0,No,Control,Phenotype unknown,Control,Control,Antecubital fossa,tape
9,WBS017,F,1993,Chinese,China,47,162,Non-drinker,No,No,WT,0,0,0,0,No,Control,Phenotype unknown,Control,Control,Antecubital fossa,tape


**Make external metadata columns Qiita-safe**

In [33]:
find_problem_headers()

Index(['Weight (kg)', 'Height (cm)', 'Blomia tropicalis (dust mite)',
       'Dermatophagoides pteronyssinus (dust mite)',
       'Elaeis guineensis (oil palm pollen)', 'Curvularia spp. (fungus)',
       'Skin Prick Test (≥3+)'],
      dtype='object')


One of these ('Skin Prick Test (≥3+)') is too complicated to be automatically scrubbed.

In [34]:
g_working_df.rename(columns={'Skin Prick Test (≥3+)': 'Skin_Prick_Test_gte3plus'}, inplace=True)

In [35]:
find_problem_headers()

Index(['Weight (kg)', 'Height (cm)', 'Blomia tropicalis (dust mite)',
       'Dermatophagoides pteronyssinus (dust mite)',
       'Elaeis guineensis (oil palm pollen)', 'Curvularia spp. (fungus)'],
      dtype='object')


The remainder should be fixable automatically.

In [36]:
scrub_headers()

Index(['libraryid', 'gender', 'year_of_birth', 'ethnicity', 'country_of_birth',
       'weight_kg', 'height_cm', 'alcohol', 'passive_smoking',
       'presence_of_pets', 'flg_genotype', 'blomia_tropicalis_dust_mite',
       'dermatophagoides_pteronyssinus_dust_mite',
       'elaeis_guineensis_oil_palm_pollen', 'curvularia_spp_fungus',
       'skin_prick_test_gte3plus', 'asthma_status', 'ar_status', 'ad_status',
       'sample_type', 'sampling_area', 'sampling_method'],
      dtype='object')


In [37]:
summarize()

The dataframe has 94 rows and 22 columns.
 
libraryid
All 94 values are unique. The first 10 unique value(s):['WBE005' 'WBE006' 'WBE007' 'WBE008' 'WBE017' 'WBE018' 'WBE019' 'WBE020'
 'WBS016' 'WBS017'].
The current datatype is object.
 
gender
There are 3 unique value(s) in 94 total values. The unique value(s):['M' 'F' nan].
The current datatype is object.
 
year_of_birth
There are 14 unique value(s) in 94 total values. The first 10 unique value(s):['1986' '1987' '1988' '1993' '1985' '1992' '1996' '1994' '1991' '1990'].
The current datatype is object.
 
ethnicity
There are 3 unique value(s) in 94 total values. The unique value(s):['Chinese' 'Caucasian' nan].
The current datatype is object.
 
country_of_birth
There are 9 unique value(s) in 94 total values. The unique value(s):['Indonesia' 'Singapore' 'China' 'Malaysia' 'Brunei' 'Hong Kong'
 'United Kingdom' 'Poland' nan].
The current datatype is object.
 
weight_kg
There are 30 unique value(s) in 94 total values. The first 10 unique val

**Clean up contents of external metadata**

Let's start by checking out the presumed unique identifier.

In [38]:
explore_col("libraryid")

The appropriate Pandas data type for the 'libraryid' column is `category`. This data type is efficient for columns that contain a limited number of unique values, which is the case here, as 'libraryid' appears to represent distinct identifiers for different libraries or study samples.

### Hypothesis:
The 'libraryid' values likely represent unique identifiers assigned to various samples or subjects in a biological or clinical study, possibly related to a database of specimens or participants.

### Invalid or Unexpected Values:
Upon reviewing the provided set, there are a few entries with trailing spaces (e.g., 'WOS012 ', 'WOS018 ', 'WOS015 ', etc.). These extra spaces could lead to inconsistencies when performing operations like filtering or merging. Additionally, one entry has an asterisk ('WBE025*'), which could indicate either a typographical error or a special status that should be clarified. It's advisable to clean these entries by stripping whitespace and addressing the asterisk.

I don't like those trailing spaces.  In fact, I don't want trailing spaces in any column.

In [39]:
find_problem_records()

,libraryid,gender,year_of_birth,ethnicity,country_of_birth,weight_kg,height_cm,alcohol,passive_smoking,presence_of_pets,flg_genotype,blomia_tropicalis_dust_mite,dermatophagoides_pteronyssinus_dust_mite,elaeis_guineensis_oil_palm_pollen,curvularia_spp_fungus,skin_prick_test_gte3plus,asthma_status,ar_status,ad_status,sample_type,sampling_area,sampling_method
81,WBS011,M,1989,Caucasian,United Kingdom,72,183,Occasionally,No,No,-,Indeterminate,Indeterminate,Indeterminate,Indeterminate,Indeterminate,Control,Control,Control,Control,Retroauricular crease,swab
83,WOS016,M,1963,Caucasian,Poland,82,189,Occasionally,No,No,-,Indeterminate,Indeterminate,Indeterminate,Indeterminate,Indeterminate,Control,Control,Control,Control,Retroauricular crease,swab
84,WOS017,M,1963,Caucasian,Poland,82,189,Occasionally,No,No,-,Indeterminate,Indeterminate,Indeterminate,Indeterminate,Indeterminate,Control,Control,Control,Control,Retroauricular crease,tape
85,WOS010,M,1963,Caucasian,Poland,82,189,Occasionally,No,No,-,Indeterminate,Indeterminate,Indeterminate,Indeterminate,Indeterminate,Control,Control,Control,Control,(Left) Antecubital Fossa,swab
86,WOS012,M,1963,Caucasian,Poland,82,189,Occasionally,No,No,-,Indeterminate,Indeterminate,Indeterminate,Indeterminate,Indeterminate,Control,Control,Control,Control,(Left) Antecubital Fossa,tape
87,WOS014,M,1963,Caucasian,Poland,82,189,Occasionally,No,No,-,Indeterminate,Indeterminate,Indeterminate,Indeterminate,Indeterminate,Control,Control,Control,Control,(Left) Antecubital Fossa,cup scrub
88,WOS011,M,1963,Caucasian,Poland,82,189,Occasionally,No,No,-,Indeterminate,Indeterminate,Indeterminate,Indeterminate,Indeterminate,Control,Control,Control,Control,(Right) Antecubital Fossa,swab
89,WOS013,M,1963,Caucasian,Poland,82,189,Occasionally,No,No,-,Indeterminate,Indeterminate,Indeterminate,Indeterminate,Indeterminate,Control,Control,Control,Control,(Right) Antecubital Fossa,tape
90,WOS015,M,1963,Caucasian,Poland,82,189,Occasionally,No,No,-,Indeterminate,Indeterminate,Indeterminate,Indeterminate,Indeterminate,Control,Control,Control,Control,(Right) Antecubital Fossa,cup scrub
91,WOS018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gloves,NaN,tape


In [40]:
scrub_problem_records()

Problem records scrubbed.


In [41]:
find_problem_records()

No problem records found.


**Set the two qiimp guide columns**

The host shorthand for anything with a gender is human, and for anything else is 'control surface'.

In [42]:
g_working_df['hosttype_shorthand'] = g_working_df['gender'].apply(lambda x: 'human' if pd.notna(x) else 'control surface')

The sample shorthand is based on the 'sampling_area' and the 'sample_type'.  If sampling_area is anything with 'Antecubital fossa' in it, it is 'antecubital fossa'.  If it is 'Retroauricular crease', it is 'retroauricular crease'.  If sampling_srea is NaN, then if sample_type is 'Gloves', the sampletype_shorthand is 'control glove' and otherwise it is 'control surface'. That should cover all the cases.

In [43]:
g_working_df['sampletype_shorthand'] = g_working_df.apply(
    lambda row: 'antecubital fossa' if pd.notna(row['sampling_area']) and 'antecubital fossa' in str(row['sampling_area']).lower() else
                 'retroauricular crease' if row['sampling_area'] == 'Retroauricular crease' else
                 'control glove' if pd.isna(row['sampling_area']) and row['sample_type'] == 'Gloves' else
                 'control surface' if pd.isna(row['sampling_area']) else
                 None,
    axis=1
)

In [44]:
g_working_df[g_working_df['sampletype_shorthand'].isna()]

,libraryid,gender,year_of_birth,ethnicity,country_of_birth,weight_kg,height_cm,alcohol,passive_smoking,presence_of_pets,flg_genotype,blomia_tropicalis_dust_mite,dermatophagoides_pteronyssinus_dust_mite,elaeis_guineensis_oil_palm_pollen,curvularia_spp_fungus,skin_prick_test_gte3plus,asthma_status,ar_status,ad_status,sample_type,sampling_area,sampling_method,hosttype_shorthand,sampletype_shorthand


In [45]:
g_working_df

,libraryid,gender,year_of_birth,ethnicity,country_of_birth,weight_kg,height_cm,alcohol,passive_smoking,presence_of_pets,flg_genotype,blomia_tropicalis_dust_mite,dermatophagoides_pteronyssinus_dust_mite,elaeis_guineensis_oil_palm_pollen,curvularia_spp_fungus,skin_prick_test_gte3plus,asthma_status,ar_status,ad_status,sample_type,sampling_area,sampling_method,hosttype_shorthand,sampletype_shorthand
0,WBE005,M,1986,Chinese,Indonesia,Not collected,Not collected,Non-drinker,No,No,WT,0,0,0,0,No,Control,Control,Control,Control,Antecubital fossa,tape,human,antecubital fossa
1,WBE006,M,1986,Chinese,Indonesia,Not collected,Not collected,Non-drinker,No,No,WT,0,0,0,0,No,Control,Control,Control,Control,Antecubital fossa,tape,human,antecubital fossa
2,WBE007,F,1987,Chinese,Singapore,47,Not collected,Non-drinker,No,No,WT,0,0,0,0,No,Indeterminate,Indeterminate,Control,Control,Antecubital fossa,tape,human,antecubital fossa
3,WBE008,F,1987,Chinese,Singapore,47,Not collected,Non-drinker,No,No,WT,0,0,0,0,No,Indeterminate,Indeterminate,Control,Control,Antecubital fossa,tape,human,antecubital fossa
4,WBE017,M,1988,Chinese,China,63,178,Occasionally,Yes,No,S1515X,0,0,0,0,No,Control,Indeterminate,Control,Control,Antecubital fossa,tape,human,antecubital fossa
5,WBE018,M,1988,Chinese,China,63,178,Occasionally,Yes,No,S1515X,0,0,0,0,No,Control,Indeterminate,Control,Control,Antecubital fossa,tape,human,antecubital fossa
6,WBE019,M,1986,Chinese,Singapore,53,163,Occasionally,No,0,WT,0,0,0,0,No,Control,Indeterminate,Control,Control,Antecubital fossa,tape,human,antecubital fossa
7,WBE020,M,1986,Chinese,Singapore,53,163,Occasionally,No,0,WT,0,0,0,0,No,Control,Indeterminate,Control,Control,Antecubital fossa,tape,human,antecubital fossa
8,WBS016,F,1993,Chinese,China,47,162,Non-drinker,No,No,WT,0,0,0,0,No,Control,Phenotype unknown,Control,Control,Antecubital fossa,tape,human,antecubital fossa
9,WBS017,F,1993,Chinese,China,47,162,Non-drinker,No,No,WT,0,0,0,0,No,Control,Phenotype unknown,Control,Control,Antecubital fossa,tape,human,antecubital fossa


In [46]:
ext_metadata_df = g_working_df.copy()

#### Examine the Qiita metadata

In [47]:
load_df(qiita_metadata_fp, sep_name="tab", override=True)

,sample_name,biosamplemodel,center_name,collection_date,ebi_metadata_retrieved,geo_loc_name,insdc_secondary_accession,isolation_source,lat_lon,ncbi_submission_package,organism,qiita_study_id,ref_biomaterial,rel_to_oxygen,samp_collect_device,samp_mat_process,samp_size,sample_accession,sample_alias,sample_title,sample_title_specific,scientific_name,secondary_sample_accession,source_material_id,study_alias,study_title,tax_id,taxon_id
0,15612.SAMN03398500,Metagenome or environmental,Genome Institrute of Singapore,not applicable,TRUE,not applicable,SRS892108,not applicable,not applicable,Metagenome.environmental.1.0,human skin metagenome,15612,not applicable,not applicable,not applicable,not applicable,not applicable,SAMN03398500,WBS004,Metagenome or environmental sample from human ...,Metagenome or environmental sample from human ...,human skin metagenome,SRS892108,WBS004,PRJNA277905,Human Skin Microbiome Metagenome,539655,539655
1,15612.SAMN03398501,Metagenome or environmental,Genome Institrute of Singapore,not applicable,TRUE,not applicable,SRS892107,not applicable,not applicable,Metagenome.environmental.1.0,human skin metagenome,15612,not applicable,not applicable,not applicable,not applicable,not applicable,SAMN03398501,WBS008,Metagenome or environmental sample from human ...,Metagenome or environmental sample from human ...,human skin metagenome,SRS892107,WBS008,PRJNA277905,Human Skin Microbiome Metagenome,539655,539655
2,15612.SAMN03398502,Metagenome or environmental,Genome Institrute of Singapore,not applicable,TRUE,not applicable,SRS892106,not applicable,not applicable,Metagenome.environmental.1.0,human skin metagenome,15612,not applicable,not applicable,not applicable,not applicable,not applicable,SAMN03398502,WBS005,Metagenome or environmental sample from human ...,Metagenome or environmental sample from human ...,human skin metagenome,SRS892106,WBS005,PRJNA277905,Human Skin Microbiome Metagenome,539655,539655
3,15612.SAMN03398503,Metagenome or environmental,Genome Institrute of Singapore,not applicable,TRUE,not applicable,SRS892104,not applicable,not applicable,Metagenome.environmental.1.0,human skin metagenome,15612,not applicable,not applicable,not applicable,not applicable,not applicable,SAMN03398503,WBS009,Metagenome or environmental sample from human ...,Metagenome or environmental sample from human ...,human skin metagenome,SRS892104,WBS009,PRJNA277905,Human Skin Microbiome Metagenome,539655,539655
4,15612.SAMN03398504,Metagenome or environmental,Genome Institrute of Singapore,not applicable,TRUE,not applicable,SRS892002,not applicable,not applicable,Metagenome.environmental.1.0,human skin metagenome,15612,not applicable,not applicable,not applicable,not applicable,not applicable,SAMN03398504,WBS006,Metagenome or environmental sample from human ...,Metagenome or environmental sample from human ...,human skin metagenome,SRS892002,WBS006,PRJNA277905,Human Skin Microbiome Metagenome,539655,539655
5,15612.SAMN03398505,Metagenome or environmental,Genome Institrute of Singapore,not applicable,TRUE,not applicable,SRS892100,not applicable,not applicable,Metagenome.environmental.1.0,human skin metagenome,15612,not applicable,not applicable,not applicable,not applicable,not applicable,SAMN03398505,WBS010,Metagenome or environmental sample from human ...,Metagenome or environmental sample from human ...,human skin metagenome,SRS892100,WBS010,PRJNA277905,Human Skin Microbiome Metagenome,539655,539655
6,15612.SAMN03398506,Metagenome or environmental,Genome Institrute of Singapore,not applicable,TRUE,not applicable,SRS892101,not applicable,not applicable,Metagenome.environmental.1.0,human skin metagenome,15612,not applicable,not applicable,not applicable,not applicable,not applicable,SAMN03398506,WBS007,Metagenome or environmental sample from human ...,Metagenome or environmental sample from human ...,human skin metagenome,SRS892101,WBS007,PRJNA277905,Human Skin Microbiome Metagenome,539655,539655
7,15612.SAMN03398507,Metagenom

In [48]:
summarize()

The dataframe has 106 rows and 28 columns.
 
sample_name
All 106 values are unique. The first 10 unique value(s):['15612.SAMN03398500' '15612.SAMN03398501' '15612.SAMN03398502'
 '15612.SAMN03398503' '15612.SAMN03398504' '15612.SAMN03398505'
 '15612.SAMN03398506' '15612.SAMN03398507' '15612.SAMN03398508'
 '15612.SAMN03398509'].
The current datatype is object.
 
biosamplemodel
There are 1 unique value(s) in 106 total values. The unique value(s):['Metagenome or environmental'].
The current datatype is object.
 
center_name
There are 1 unique value(s) in 106 total values. The unique value(s):['Genome Institrute of Singapore'].
The current datatype is object.
 
collection_date
There are 1 unique value(s) in 106 total values. The unique value(s):['not applicable'].
The current datatype is object.
 
ebi_metadata_retrieved
There are 1 unique value(s) in 106 total values. The unique value(s):['TRUE'].
The current datatype is object.
 
geo_loc_name
There are 1 unique value(s) in 106 total values

In [49]:
explore_col("center_name")

The appropriate Pandas data type for the 'center_name' column is `category`. This is suitable because it contains a limited number of unique values—in this case, just one unique center name. Using the `category` type will optimize memory usage and performance for operations on this column.

### Hypothesis:
The value in the 'center_name' column likely represents the name of the research or sequencing center from which the samples or data were collected. In this case, it indicates that all data in this dataframe is sourced from the "Genome Institute of Singapore."

### Invalid or Unexpected Values:
The only entry provided is 'Genome Institrute of Singapore,' which contains a typographical error: "Institrute" should likely be "Institute." This error may lead to confusion or misidentification of the center in analyses or reporting. It should be corrected for accuracy and consistency. If there are other entries in the future, they should also be checked for similar errors or unexpected vari

In [50]:
g_working_df['center_name'] = g_working_df['center_name'].replace('Genome Institrute of Singapore', 'Genome Institute of Singapore')

I know the libraryid column of the external metadata should be matched with the qiita metadata's sample_alias column, so let's take a look at the latter.

In [51]:
explore_col("sample_alias")

The appropriate Pandas data type for the 'sample_alias' column is `category`. This data type is suitable because there is a finite set of unique sample identifiers, which can be efficiently represented as categorical values.

### Hypothesis:
The values in the 'sample_alias' column likely represent unique identifiers or aliases for biological samples in a study. These identifiers are often used to label and reference samples in data analyses, allowing researchers to track and manage different specimens easily.

### Invalid or Unexpected Values:
Upon reviewing the provided set, there are no immediately visible invalid or unexpected values. All entries follow a consistent pattern, typically alphanumeric with a specific prefix (e.g., 'WBU', 'WBS', 'WBE', 'WOS'). However, it would be prudent to check for any duplicates or ensure that these aliases conform to any expected naming conventions within the context of the study. Additionally, if there are any expected prefixes or suffixes, or if t

#### Merge the qiita and external metadata

First check for columns that overlap.

In [52]:
qiimp.find_common_df_cols(g_working_df, ext_metadata_df)

[]

I don't understand how this existing qiita metadata doesn't already have a sample_type column, but I guess it doesn't, so ok; we can go ahead and merge.

In [53]:
merged_df = qiimp.merge_one_to_one_metadata(g_working_df, ext_metadata_df, "sample_alias", "libraryid", "qiita", "external", "outer")
merged_df

,sample_name,biosamplemodel,center_name,collection_date,ebi_metadata_retrieved,geo_loc_name,insdc_secondary_accession,isolation_source,lat_lon,ncbi_submission_package,organism,qiita_study_id,ref_biomaterial,rel_to_oxygen,samp_collect_device,samp_mat_process,samp_size,sample_accession,sample_alias,sample_title,sample_title_specific,scientific_name,secondary_sample_accession,source_material_id,study_alias,study_title,tax_id,taxon_id,libraryid,gender,year_of_birth,ethnicity,country_of_birth,weight_kg,height_cm,alcohol,passive_smoking,presence_of_pets,flg_genotype,blomia_tropicalis_dust_mite,dermatophagoides_pteronyssinus_dust_mite,elaeis_guineensis_oil_palm_pollen,curvularia_spp_fungus,skin_prick_test_gte3plus,asthma_status,ar_status,ad_status,sample_type,sampling_area,sampling_method,hosttype_shorthand,sampletype_shorthand
0,15612.SAMN03398557,Metagenome or environmental,Genome Institute of Singapore,not applicable,TRUE,not applicable,SRS892051,not applicable,not applicable,Metagenome.environmental.1.0,human skin metagenome,15612,not applicable,not applicable,not applicable,not applicable,not applicable,SAMN03398557,WBE003,Metagenome or environmental sample from human ...,Metagenome or environmental sample from human ...,human skin metagenome,SRS892051,WBE003,PRJNA277905,Human Skin Microbiome Metagenome,539655,539655,WBE003,M,1987,Chinese,Singapore,84,182,Non-drinker,No,Yes,WT,3+,3+,0,0,Yes,Indeterminate,Case,Case,Case,Antecubital fossa,tape,human,antecubital fossa
1,15612.SAMN03398558,Metagenome or environmental,Genome Institute of Singapore,not applicable,TRUE,not applicable,SRS892050,not applicable,not applicable,Metagenome.environmental.1.0,human skin metagenome,15612,not applicable,not applicable,not applicable,not applicable,not applicable,SAMN03398558,WBE004,Metagenome or environmental sample from human ...,Metagenome or environmental sample from human ...,human skin metagenome,SRS892050,WBE004,PRJNA277905,Human Skin Microbiome Metagenome,539655,539655,WBE004,M,1987,Chinese,Singapore,84,182,Non-drinker,No,Yes,WT,3+,3+,0,0,Yes,Indeterminate,Case,Case,Case,Antecubital fossa,tape,human,antecubital fossa
2,15612.SAMN03398527,Metagenome or environmental,Genome Institute of Singapore,not applicable,TRUE,not applicable,SRS892082,not applicable,not applicable,Metagenome.environmental.1.0,human skin metagenome,15612,not applicable,not applicable,not applicable,not applicable,not applicable,SAMN03398527,WBE005,Metagenome or environmental sample from human ...,Metagenome or environmental sample from human ...,human skin metagenome,SRS892082,WBE005,PRJNA277905,Human Skin Microbiome Metagenome,539655,539655,WBE005,M,1986,Chinese,Indonesia,Not collected,Not collected,Non-drinker,No,No,WT,0,0,0,0,No,Control,Control,Control,Control,Antecubital fossa,tape,human,antecubital fossa
3,15612.SAMN03398528,Metagenome or environmental,Genome Institute of Singapore,not applicable,TRUE,not applicable,SRS892080,not applicable,not applicable,Metagenome.environmental.1.0,human skin metagenome,15612,not applicable,not applicable,not applicable,not applicable,not applicable,SAMN03398528,WBE006,Metagenome or environmental sample from human ...,Metagenome or environmental sample from human ...,human skin metagenome,SRS892080,WBE006,PRJNA277905,Human Skin Microbiome Metagenome,539655,539655,WBE006,M,1986,Chinese,Indonesia,Not collected,Not collected,Non-drinker,No,No,WT,0,0,0,0,No,Control,Control,Control,Control,Antecubital fossa,tape,human,antecubital fossa
4,15612.SAMN03398529,Metagenome or environmental,Genome Institute of Singapore,not applicable,TRUE,not applicable,SRS892079,not applicable,not applicable,Metagenome.environmental.1.0,human skin metagenome,15612,not applicable,not applicable,not applicable,not applicable,not applicable,SAMN03398529,WBE007,Metagenome or environmental sample from human ...,Metagenome or environmental sample from human ...,human skin metagenome,SRS892079,WBE007,PRJNA277905,Human Skin Microbiome Metagenome,539655,539655,WBE

Look for records that occur in the external metadata but not in the qiita metadata:

In [54]:
merged_df[merged_df["sample_name"].isna()]

,sample_name,biosamplemodel,center_name,collection_date,ebi_metadata_retrieved,geo_loc_name,insdc_secondary_accession,isolation_source,lat_lon,ncbi_submission_package,organism,qiita_study_id,ref_biomaterial,rel_to_oxygen,samp_collect_device,samp_mat_process,samp_size,sample_accession,sample_alias,sample_title,sample_title_specific,scientific_name,secondary_sample_accession,source_material_id,study_alias,study_title,tax_id,taxon_id,libraryid,gender,year_of_birth,ethnicity,country_of_birth,weight_kg,height_cm,alcohol,passive_smoking,presence_of_pets,flg_genotype,blomia_tropicalis_dust_mite,dermatophagoides_pteronyssinus_dust_mite,elaeis_guineensis_oil_palm_pollen,curvularia_spp_fungus,skin_prick_test_gte3plus,asthma_status,ar_status,ad_status,sample_type,sampling_area,sampling_method,hosttype_shorthand,sampletype_shorthand
23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WBE025*,F,1993,Chinese,Singapore,50,161.5,Occasionally,Yes,Yes,WT,3+,3+,0,0,Yes,Indeterminate,Indeterminate,Case,Case,Antecubital fossa,tape,human,antecubital fossa


Ok, so there's one, and it's the one with the library id that has an asterisk on it (maybe not surprising).  

Conversely, are there records that occur in the qiita metadata but not in the external metadata?

In [55]:
merged_df[merged_df["libraryid"].isna()]

,sample_name,biosamplemodel,center_name,collection_date,ebi_metadata_retrieved,geo_loc_name,insdc_secondary_accession,isolation_source,lat_lon,ncbi_submission_package,organism,qiita_study_id,ref_biomaterial,rel_to_oxygen,samp_collect_device,samp_mat_process,samp_size,sample_accession,sample_alias,sample_title,sample_title_specific,scientific_name,secondary_sample_accession,source_material_id,study_alias,study_title,tax_id,taxon_id,libraryid,gender,year_of_birth,ethnicity,country_of_birth,weight_kg,height_cm,alcohol,passive_smoking,presence_of_pets,flg_genotype,blomia_tropicalis_dust_mite,dermatophagoides_pteronyssinus_dust_mite,elaeis_guineensis_oil_palm_pollen,curvularia_spp_fungus,skin_prick_test_gte3plus,asthma_status,ar_status,ad_status,sample_type,sampling_area,sampling_method,hosttype_shorthand,sampletype_shorthand
22,15612.SAMN03398595,Metagenome or environmental,Genome Institute of Singapore,not applicable,TRUE,not applicable,SRS892013,not applicable,not applicable,Metagenome.environmental.1.0,human skin metagenome,15612,not applicable,not applicable,not applicable,not applicable,not applicable,SAMN03398595,WBE025,Metagenome or environmental sample from human ...,Metagenome or environmental sample from human ...,human skin metagenome,SRS892013,WBE025,PRJNA277905,Human Skin Microbiome Metagenome,539655,539655,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,15612.SAMN03398508,Metagenome or environmental,Genome Institute of Singapore,not applicable,TRUE,not applicable,SRS892102,not applicable,not applicable,Metagenome.environmental.1.0,human skin metagenome,15612,not applicable,not applicable,not applicable,not applicable,not applicable,SAMN03398508,WBS002,Metagenome or environmental sample from human ...,Metagenome or environmental sample from human ...,human skin metagenome,SRS892102,WBS002,PRJNA277905,Human Skin Microbiome Metagenome,539655,539655,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,15612.SAMN03398510,Metagenome or environmental,Genome Institute of Singapore,not applicable,TRUE,not applicable,SRS892098,not applicable,not applicable,Metagenome.environmental.1.0,human skin metagenome,15612,not applicable,not applicable,not applicable,not applicable,not applicable,SAMN03398510,WBS003,Metagenome or environmental sample from human ...,Metagenome or environmental sample from human ...,human skin metagenome,SRS892098,WBS003,PRJNA277905,Human Skin Microbiome Metagenome,539655,539655,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,15612.SAMN03398500,Metagenome or environmental,Genome Institute of Singapore,not applicable,TRUE,not applicable,SRS892108,not applicable,not applicable,Metagenome.environmental.1.0,human skin metagenome,15612,not applicable,not applicable,not applicable,not applicable,not applicable,SAMN03398500,WBS004,Metagenome or environmental sample from human ...,Metagenome or environmental sample from human ...,human skin metagenome,SRS892108,WBS004,PRJNA277905,Human Skin Microbiome Metagenome,539655,539655,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,15612.SAMN03398502,Metagenome or environmental,Genome Institute of Singapore,not applicable,TRUE,not applicable,SRS892106,not applicable,not applicable,Metagenome.environmental.1.0,human skin metagenome,15612,not applicable,not applicable,not applicable,not applicable,not applicable,SAMN03398502,WBS005,Metagenome or environmental sample from human ...,Metagenome or environmental sample from human ...,human skin metagenome,SRS892106,WBS005,PRJNA277905,Human Skin Microbiome Metagenome,539655,539655,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,15612.SAMN03398504,Metagenome or environmental,Genome Institute of Singapore,not applicable,TRUE,not applicable,SRS892002,not applicable,not applicable,Metagen

Hmm, what were the unique libraryids in the external metadata again?

In [56]:
ext_metadata_df["libraryid"].unique()

array(['WBE005', 'WBE006', 'WBE007', 'WBE008', 'WBE017', 'WBE018',
       'WBE019', 'WBE020', 'WBS016', 'WBS017', 'WBS018', 'WBS019',
       'WBU001', 'WBU002', 'WBU003', 'WBU004', 'WBU007', 'WBU008',
       'WBU009', 'WBU010', 'WBU011', 'WBU012', 'WBU013', 'WBU014',
       'WBU015', 'WBU016', 'WBU017', 'WBU018', 'WBU019', 'WBU020',
       'WBE042', 'WBE043', 'WBS012', 'WBS013', 'WBS014', 'WBS015',
       'WBS020', 'WBS021', 'WBS022', 'WBS023', 'WBE003', 'WBE004',
       'WBE009', 'WBE010', 'WBE011', 'WBE012', 'WBE013', 'WBE014',
       'WBE015', 'WBE016', 'WBE021', 'WBE022', 'WBE023', 'WBE024',
       'WBE025*', 'WBE026', 'WBE027', 'WBE028', 'WBE029', 'WBE030',
       'WBE031', 'WBE032', 'WBE033', 'WBE034', 'WBE035', 'WBE036',
       'WBE037', 'WBE038', 'WBE039', 'WBE040', 'WBE041', 'WBE044',
       'WBE045', 'WBE046', 'WBE047', 'WBE048', 'WBE049', 'WBU005',
       'WBU006', 'WBS009', 'WBS008', 'WBS011', 'WBS010', 'WOS016',
       'WOS017', 'WOS010', 'WOS012', 'WOS014', 'WOS011', 'WOS

In [57]:
'WBS003' in ext_metadata_df["libraryid"].unique()

False

Okay, I guess ...

In [58]:
g_working_df = merged_df.copy()

**Find columns in the external metadata that might be touched by qiimp**

Rename them if they are in danger of being overwritten.

In [59]:
qiimp_cols = qiimp.get_reserved_cols(g_working_df, {})

In [60]:
# the qiimp guide columns and the sample_name column are expected by qiimp and won't be altered
expected_shared_cols = [qiimp.HOSTTYPE_SHORTHAND_KEY, qiimp.SAMPLETYPE_SHORTHAND_KEY, "sample_name"]
qiimp.find_common_col_names(g_working_df.columns, qiimp_cols, expected_shared_cols, expected_shared_cols)

['geo_loc_name',
 'sample_type',
 'taxon_id',
 'collection_date',
 'scientific_name']

In [61]:
summarize_col("geo_loc_name")

geo_loc_name
There are 2 unique value(s) in 107 total values. The unique value(s):['not applicable' nan].
The current datatype is object.
 


In [62]:
summarize_col("scientific_name")

scientific_name
There are 2 unique value(s) in 107 total values. The unique value(s):['human skin metagenome' nan].
The current datatype is object.
 


In [63]:
summarize_col("sample_type")

sample_type
There are 7 unique value(s) in 107 total values. The unique value(s):['Case' 'Control' nan 'Atopic Control' 'Gloves' 'Bench Top' 'Door Knob'].
The current datatype is object.
 


Ok, the 'sample_type' column contains external values worth preserving, so let's rename that column so it is saved.

In [64]:
g_working_df.rename(columns={'sample_type': 'sample_type_external'}, inplace=True)

**Look for standard columns we could fill with the info in the metadata**

In [65]:
qiimp_cols

['collection_date',
 'collection_timestamp',
 'country',
 'description',
 'dna_extracted',
 'elevation',
 'empo_1',
 'empo_2',
 'empo_3',
 'empo_4',
 'env_biome',
 'env_feature',
 'env_material',
 'env_package',
 'geo_loc_name',
 'host_age',
 'host_age_units',
 'host_body_habitat',
 'host_body_mass_index',
 'host_body_product',
 'host_body_site',
 'host_common_name',
 'host_height',
 'host_height_units',
 'host_life_stage',
 'host_scientific_name',
 'host_subject_id',
 'host_taxid',
 'host_weight',
 'host_weight_units',
 'hosttype_shorthand',
 'latitude',
 'longitude',
 'physical_specimen_location',
 'physical_specimen_remaining',
 'qc_note',
 'qiita_sample_type',
 'sample_name',
 'sample_type',
 'sampletype_shorthand',
 'scientific_name',
 'sex',
 'taxon_id',
 'title']

In [66]:
summarize()

The dataframe has 107 rows and 52 columns.
 
sample_name
All 107 values are unique. The first 10 unique value(s):['15612.SAMN03398557' '15612.SAMN03398558' '15612.SAMN03398527'
 '15612.SAMN03398528' '15612.SAMN03398529' '15612.SAMN03398530'
 '15612.SAMN03398559' '15612.SAMN03398560' '15612.SAMN03398561'
 '15612.SAMN03398562'].
The current datatype is object.
 
biosamplemodel
There are 2 unique value(s) in 107 total values. The unique value(s):['Metagenome or environmental' nan].
The current datatype is object.
 
center_name
There are 2 unique value(s) in 107 total values. The unique value(s):['Genome Institute of Singapore' nan].
The current datatype is object.
 
collection_date
There are 2 unique value(s) in 107 total values. The unique value(s):['not applicable' nan].
The current datatype is object.
 
ebi_metadata_retrieved
There are 2 unique value(s) in 107 total values. The unique value(s):['TRUE' nan].
The current datatype is object.
 
geo_loc_name
There are 2 unique value(s) in 1

Well, we can definitely set the standard 'sex' column from the 'gender' column, and the 'host_weight' from the 'weight_kg' column, and the 'host_height' from the 'height_cm' column. With just the year of birth, we don't have enough to fill in the age, though, and I don't want to try to get host_life_stage based on just the birth year, either.

In [67]:
g_working_df['sex'] = g_working_df['gender'].apply(qiimp.standardize_input_sex)

In [68]:
g_working_df['host_weight'] = g_working_df['weight_kg']
g_working_df['host_height'] = g_working_df['height_cm']

**Set TRPCA-specific columns from the metadata**

For this study, they are all conditional on various sample-specific metadata columns.

Wait, first I'm going to remove that record that doesn't have a sample_name, since it can't go into qiita anyway.

In [69]:
g_working_df.shape

(107, 55)

ask("Write code to remove the record that has a NaN 'sample_name' from the dataframe.")

run_suggestion()

In [70]:
g_working_df.dropna(subset=['sample_name'], inplace=True)

In [71]:
g_working_df.shape

(106, 55)

Ok, lesional status should be not provided for human records and not applicable for others.  There are those samples that we don't have any external metadata for, and they *might* be human, but I don't know; I think I'm going to just treat them as 'not applicable's.

In [72]:
g_working_df['lesional_status'] = g_working_df['gender'].apply(lambda x: 'not applicable' if pd.isna(x) else 'not provided')

In [73]:
g_working_df

,sample_name,biosamplemodel,center_name,collection_date,ebi_metadata_retrieved,geo_loc_name,insdc_secondary_accession,isolation_source,lat_lon,ncbi_submission_package,organism,qiita_study_id,ref_biomaterial,rel_to_oxygen,samp_collect_device,samp_mat_process,samp_size,sample_accession,sample_alias,sample_title,sample_title_specific,scientific_name,secondary_sample_accession,source_material_id,study_alias,study_title,tax_id,taxon_id,libraryid,gender,year_of_birth,ethnicity,country_of_birth,weight_kg,height_cm,alcohol,passive_smoking,presence_of_pets,flg_genotype,blomia_tropicalis_dust_mite,dermatophagoides_pteronyssinus_dust_mite,elaeis_guineensis_oil_palm_pollen,curvularia_spp_fungus,skin_prick_test_gte3plus,asthma_status,ar_status,ad_status,sample_type_external,sampling_area,sampling_method,hosttype_shorthand,sampletype_shorthand,sex,host_weight,host_height,lesional_status
0,15612.SAMN03398557,Metagenome or environmental,Genome Institute of Singapore,not applicable,TRUE,not applicable,SRS892051,not applicable,not applicable,Metagenome.environmental.1.0,human skin metagenome,15612,not applicable,not applicable,not applicable,not applicable,not applicable,SAMN03398557,WBE003,Metagenome or environmental sample from human ...,Metagenome or environmental sample from human ...,human skin metagenome,SRS892051,WBE003,PRJNA277905,Human Skin Microbiome Metagenome,539655,539655,WBE003,M,1987,Chinese,Singapore,84,182,Non-drinker,No,Yes,WT,3+,3+,0,0,Yes,Indeterminate,Case,Case,Case,Antecubital fossa,tape,human,antecubital fossa,male,84,182,not provided
1,15612.SAMN03398558,Metagenome or environmental,Genome Institute of Singapore,not applicable,TRUE,not applicable,SRS892050,not applicable,not applicable,Metagenome.environmental.1.0,human skin metagenome,15612,not applicable,not applicable,not applicable,not applicable,not applicable,SAMN03398558,WBE004,Metagenome or environmental sample from human ...,Metagenome or environmental sample from human ...,human skin metagenome,SRS892050,WBE004,PRJNA277905,Human Skin Microbiome Metagenome,539655,539655,WBE004,M,1987,Chinese,Singapore,84,182,Non-drinker,No,Yes,WT,3+,3+,0,0,Yes,Indeterminate,Case,Case,Case,Antecubital fossa,tape,human,antecubital fossa,male,84,182,not provided
2,15612.SAMN03398527,Metagenome or environmental,Genome Institute of Singapore,not applicable,TRUE,not applicable,SRS892082,not applicable,not applicable,Metagenome.environmental.1.0,human skin metagenome,15612,not applicable,not applicable,not applicable,not applicable,not applicable,SAMN03398527,WBE005,Metagenome or environmental sample from human ...,Metagenome or environmental sample from human ...,human skin metagenome,SRS892082,WBE005,PRJNA277905,Human Skin Microbiome Metagenome,539655,539655,WBE005,M,1986,Chinese,Indonesia,Not collected,Not collected,Non-drinker,No,No,WT,0,0,0,0,No,Control,Control,Control,Control,Antecubital fossa,tape,human,antecubital fossa,male,Not collected,Not collected,not provided
3,15612.SAMN03398528,Metagenome or environmental,Genome Institute of Singapore,not applicable,TRUE,not applicable,SRS892080,not applicable,not applicable,Metagenome.environmental.1.0,human skin metagenome,15612,not applicable,not applicable,not applicable,not applicable,not applicable,SAMN03398528,WBE006,Metagenome or environmental sample from human ...,Metagenome or environmental sample from human ...,human skin metagenome,SRS892080,WBE006,PRJNA277905,Human Skin Microbiome Metagenome,539655,539655,WBE006,M,1986,Chinese,Indonesia,Not collected,Not collected,Non-drinker,No,No,WT,0,0,0,0,No,Control,Control,Control,Control,Antecubital fossa,tape,human,antecubital fossa,male,Not collected,Not collected,not provided
4,15612.SAMN03398529,Metagenome or environmental,Genome Institute of Singapore,not applicable,TRUE,not applicable,SRS892079,not applicable,not applicable,Metagenome.environmental.1.0,human skin metagenome,15612,not applicable,not applicable,not applicable,not applicable,not applicable,SAMN03398529,WBE007,Metage

In [74]:
g_working_df['has_skin_disorder'] = g_working_df['ad_status'].apply(lambda x: True if x == 'Case' else (False if x == 'Control' else 'not applicable'))

In [75]:
g_working_df['skin_disorder'] = g_working_df['ad_status'].apply(lambda x: 'atopic dermatitis' if x == 'Case' else 'not_applicable')

#### Run qiimp to extend the metadata with standard column values

In [76]:
extended_df, validation_msgs = qiimp.get_extended_metadata_from_df_and_yaml(g_working_df, study_config_fp)

In [77]:
g_working_df.shape

(106, 58)

In [78]:
extended_df.shape

(106, 94)

Were there any qc notes or validation messages?

In [79]:
extended_df[extended_df['qc_note'].notnull() & (extended_df['qc_note'] != '')]

,sample_name,ad_status,alcohol,ar_status,asthma_status,biosamplemodel,blomia_tropicalis_dust_mite,center_name,collection_date,collection_timestamp,country,country_of_birth,curvularia_spp_fungus,dermatophagoides_pteronyssinus_dust_mite,description,dna_extracted,ebi_metadata_retrieved,elaeis_guineensis_oil_palm_pollen,elevation,empo_1,empo_2,empo_3,empo_4,env_biome,env_feature,env_material,env_package,ethnicity,flg_genotype,gender,geo_loc_name,has_skin_disorder,height_cm,host_age,host_age_units,host_body_habitat,host_body_mass_index,host_body_product,host_body_site,host_common_name,host_height,host_height_units,host_life_stage,host_scientific_name,host_subject_id,host_taxid,host_weight,host_weight_units,insdc_secondary_accession,isolation_source,lat_lon,latitude,lesional_status,libraryid,longitude,ncbi_submission_package,occlusion,organism,passive_smoking,physical_specimen_location,physical_specimen_remaining,presence_of_pets,qiita_sample_type,qiita_study_id,ref_biomaterial,rel_to_oxygen,samp_collect_device,samp_mat_process,samp_size,sample_accession,sample_alias,sample_title,sample_title_specific,sample_type,sample_type_external,sampling_area,sampling_method,scientific_name,secondary_sample_accession,sex,skin_disorder,skin_prick_test_gte3plus,skin_type,source_material_id,study_alias,study_title,tax_id,taxon_id,title,weight_kg,year_of_birth,hosttype_shorthand,sampletype_shorthand,qc_note
90,15612.SAMN03398595,not applicable,not applicable,not applicable,not applicable,Metagenome or environmental,not applicable,Genome Institute of Singapore,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,TRUE,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,SRS892013,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,Metagenome.environmental.1.0,not applicable,human skin metagenome,not applicable,not applicable,not applicable,not applicable,not applicable,15612,not applicable,not applicable,not applicable,not applicable,not applicable,SAMN03398595,WBE025,Metagenome or environmental sample from human ...,Metagenome or environmental sample from human ...,not applicable,not applicable,not applicable,not applicable,human skin metagenome,SRS892013,not applicable,not_applicable,not applicable,not applicable,WBE025,PRJNA277905,Human Skin Microbiome Metagenome,539655,539655,not applicable,not applicable,not applicable,empty,empty,invalid host_type
91,15612.SAMN03398508,not applicable,not applicable,not applicable,not applicable,Metagenome or environmental,not applicable,Genome Institute of Singapore,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,TRUE,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,SRS892102,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,Metagenome.environmental.1.0,not applicable,human skin metagenome,not applicable,not applicable,not applicable,not applicable,not applicable,15612,not applicable,not applicable,not applicable,not applicable,not applicable,SAMN03398508,WBS002,Metagenome or environmen

It makes sense that all the ones without libraryids can't have their host and sample types set and thus can't be extended.

In [80]:
validation_msgs

,sample_name,field_name,error_message
0,15612.SAMN03398527,host_height,"[no definitions validate, {'anyof definition 0..."
1,15612.SAMN03398527,host_weight,"[no definitions validate, {'anyof definition 0..."
2,15612.SAMN03398528,host_height,"[no definitions validate, {'anyof definition 0..."
3,15612.SAMN03398528,host_weight,"[no definitions validate, {'anyof definition 0..."
4,15612.SAMN03398529,host_height,"[no definitions validate, {'anyof definition 0..."
5,15612.SAMN03398530,host_height,"[no definitions validate, {'anyof definition 0..."


Well, ok, so there are a bunch of fields that were set to unexpected values (in qiita, before the metadata got to me).  Maybe they should be modified, but I'm not sure what to, so I am going to ignore them and send them back to the stakeholder to deal with.

In [81]:
qiimp.write_metadata_results(extended_df, validation_msgs, '/Users/abirmingham/Desktop/trpca', "15612_merged_metadata_standardized", remove_internals=True, suppress_empty_fails=True)

**Example of using voice recognition**

Run `g_buttons` in a code cell to display the buttons.  After you record a command and it is run, the buttons will automatically move down below the new cells.

In [83]:
ask('write code to rename column field name')

g_working_df.rename(columns={
    'libraryid': 'library_id',
    'year_of_birth': 'birth_year',
    'country_of_birth': 'birth_country',
    'weight_kg': 'weight_kg',
    'height_cm': 'height_cm',
    'flg_genotype': 'genotype_flag',
    'sample_type_external': 'external_sample_type',
    'sampling_area': 'sampling_area',
    'sampling_method': 'sampling_method',
    'hosttype_shorthand': 'host_type_short',
    'sampletype_shorthand': 'sample_type_short',
    'has_skin_disorder': 'skin_disorder_flag'
}, inplace=True)


In [82]:
g_buttons